In [185]:
import primer3

# Import modules
from multiplexdesigner.primer3_port import seqtm, end_oligodg
from multiplexdesigner.primer_three import gc_content

def reverse_complement(dna):
    """
    Returns the reverse complement of a DNA sequence.
    
    Args:
        dna (str): DNA sequence string (A, T, G, C)
    
    Returns:
        str: Reverse complement of the input DNA sequence
    """
    complement = {'A': 'T', 'T': 'A', 'G': 'C', 'C': 'G'}
    
    # Handle lowercase and raise error for invalid bases
    dna = dna.upper()
    
    try:
        reverse_comp = ''.join(complement[base] for base in reversed(dna))
        return reverse_comp
    except KeyError as e:
        raise ValueError(f"Invalid DNA base: {e.args[0]}")

In [186]:
template = 'CAAAATATTCGTTTTAAGGGTAAAGAAAAAAGTTAAAAAATCTATTTACATAAAAAATAAGAACACTGATTTTTGTGAATACTGGGAACTATGAAAATACTATAGTTGAGACCTTCAATGACTTTCTAGTAACTCAGCAGCATCTCAGGGCCAAAAATTTAATCAGTGGAAAAATAGCCTCAATTCTTACCATCCACAAAATGGATCCAGACAACTGTTCAAACTGATGGGACCCACTCCATCGAGATTTCACTGTAGCTAGACCAAAATCACCTATTTTTACTGTGAGGTCTTCATGAAGAAATATATCTGAGGTGTAGTAAGTAAAGGAAAACAGTAGATCTCATTTTCCTATCAGAGCAAGCATTATGAAGAGTTTAGGTAAGAGATCTAATTTCTATAATTCTGTAATATAATATTCTTTAAAACATAGTACTTCATCTTTCCTCTTAGAGTCAATAAGTATGTCTAAAACAATGATTAGTTCTATTTAGCCTATA'

primer1 = 'AGGGCCAAAAATTTAATCAGTGGA'
primer2 = 'TCGATGGAGTGGGTCCCAT'

oligo_calc = primer3.thermoanalysis.ThermoAnalysis()

oligo_calc.set_thermo_args(
    mv_conc = 50.0,
    dv_conc = 1.5,
    dntp_conc = 0.6,
    dna_conc = 50.0,
    dmso_conc = 0.0,
    dmso_fact = 0.6,
    formamide_conc = 0.0,
    annealing_temp_c = -10,
    temp_c = 37.0,
    max_nn_length = 60,
    max_loop = 30,
    tm_method = 'santalucia',
    salt_corrections_method = 'santalucia'
)

In [187]:
seqtm(primer1)

Running oligotm


TmResult(Tm=59.1564840646144, bound=41.45683945767447)

In [188]:
oligo_calc.calc_tm(primer1)

59.1564840646144

In [189]:
seqtm(primer2)

Running oligotm


TmResult(Tm=59.99707857620194, bound=49.976271130952604)

In [190]:
oligo_calc.calc_tm(primer2)

59.99707857620194

In [191]:
round(gc_content(primer1),1)

37.5

In [192]:
round(gc_content(primer2),1)

57.9

## Hairpins

In [193]:
oligo_calc.calc_hairpin(primer1)

ThermoResult(structure_found=False, tm=0.00, dg=0.00, dh=0.00, ds=0.00)

In [194]:
oligo_calc.calc_hairpin(primer2)

ThermoResult(structure_found=True, tm=46.31, dg=-597.27, dh=-20500.00, ds=-64.17)

## Mispriming

In [205]:
oligo_calc.calc_heterodimer(primer1, template)

ThermoResult(structure_found=True, tm=24.19, dg=-7084.75, dh=-95900.00, ds=-286.36)

In [207]:
oligo_calc.calc_heterodimer(primer2, reverse_complement(template))

ThermoResult(structure_found=True, tm=15.49, dg=-7056.95, dh=-55800.00, ds=-157.16)

## Dimers

In [197]:
oligo_calc.calc_homodimer(primer1)

ThermoResult(structure_found=True, tm=6.18, dg=-6030.12, dh=-47000.00, ds=-132.10)

In [198]:
oligo_calc.calc_homodimer(primer2)

ThermoResult(structure_found=True, tm=15.49, dg=-7056.95, dh=-55800.00, ds=-157.16)

In [199]:
# "PRIMER_MAX_SELF_END_TH": 47.0
# Is this correct?
oligo_calc.calc_end_stability(primer1, primer1)

ThermoResult(structure_found=True, tm=-73.86, dg=-925.15, dh=-18500.00, ds=-56.67)

In [204]:
# "PRIMER_MAX_SELF_END_TH": 47.0
# Is this correct?
oligo_calc.calc_end_stability(primer2, primer2)

ThermoResult(structure_found=True, tm=2.96, dg=-3744.77, dh=-60600.00, ds=-183.32)

In [200]:
oligo_calc.calc_heterodimer(primer1, primer2)

ThermoResult(structure_found=True, tm=-1.93, dg=-3607.80, dh=-53000.00, ds=-159.25)

In [201]:
# "PRIMER_PAIR_MAX_COMPL_END_TH": 47.0
# Is this correct?
oligo_calc.calc_end_stability(primer1, primer2)

ThermoResult(structure_found=True, tm=-59.28, dg=-1626.78, dh=-21300.00, ds=-63.43)

## 5' and 3' stability

The stability of the primer determines its false priming efficiency. An ideal primer has a stable 5' end and an unstable 3' end.

If the primer has a stable 3' end, it will bond to a site which is complementary to it other than the target with its 5' end hanging off the edge. It may then lead to secondary bands.

The 3' end stability is the DG value of the 5 bases of primer taken from 3' end. 

Stability of the 5' termini allows for efficient bonding of the primer to the target site. This stable 5' region is called the GC Clamp. It ensures adequate binding of the primer to the template. Use of primers with optimal stability allows for the use of lower annealing temperatures without the production of secondary bands. Notice that the 3' end should not be very stable and the 5' end should have a strong GC clamp.

The GC Clamp is the DG value of the 5 bases of primer taken from 5' end. The lower this value, numerically, the more efficient is the primer.

In [202]:
round(end_oligodg(primer1, 5),1)

4.0

In [203]:
round(end_oligodg(primer2, 5),1)

4.0